# WG: Process Precip for H1

Process synthetic precipitation time series for H1 analysis

In [1]:
%matplotlib inline

In [2]:
import os
from IPython.display import display, HTML
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from scipy import stats as sstats
import sys

In [3]:
sys.path.append( "D:\Repositories\WeatherGenerator" )

In [4]:
import WG_Inputs as WGI

In [5]:
IN_DIR = r'C:\Temp\WG_Test_Out\Test1\Processed'
H1P1_ROOT = "H1P1PrecipAllReal_G"
H1P2_ROOT = "H1P2PrecipAllReal_G"
H1P3_ROOT = "H1P3PrecipAllReal_G"
PICKLE_EXT = "pickle"
OUT_ROOT = "DC_WGMN1"

### H1 CProj Period 1

Expect all of the H1 projection cases to be different. In addition, we have WGI.NUM_LOCA_GRID different locations. To do these, we need to loop through all grid locations for each one.

In [6]:
InFileName = "%s%d.%s" % ( H1P1_ROOT, WGI.LOCA_KEYS[0], PICKLE_EXT )
InFile = os.path.normpath( os.path.join( IN_DIR, InFileName ) )
H1DF = pd.read_pickle( InFile, compression='zip' )

In [7]:
RealColumns = H1DF.columns.to_list()

First do all of our set up stuff

In [8]:
OurQs = np.array( [ round( 0.05 * x, 2 ) for x in range(21) ], dtype=np.float32 )
NumQs = len( OurQs )
NumQs

21

In [9]:
StatsHdsList = list()
for iI in range(NumQs):
    cQ = OurQs[iI]
    Hdr = "%d_ptile" % round(cQ * 100.0)
    StatsHdsList.append( Hdr )
# end of for
StatsHdsList.append( "Average" )
StatsHdsList.append( "Variance" )
StatsHdsList.append( "Skew" )
StatsHdsList.append( "Kurt" )

In [10]:
DayYrList = [x for x in range(1, 367, 1)]
DayYrIndex = np.array( DayYrList, dtype=np.int32 )

In [11]:
MonthLister = [ "Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul", "Aug", 
                 "Sep", "Oct", "Nov", "Dec", "All" ]

In [12]:
gCnt = 0
for gG in WGI.LOCA_KEYS:
    InFileName = "%s%d.%s" % ( H1P1_ROOT, gG, PICKLE_EXT )
    InFile = os.path.normpath( os.path.join( IN_DIR, InFileName ) )
    H1DF = pd.read_pickle( InFile, compression='zip' )
    # collect the overall statistics
    H1StatsList = list()
    npFlat = H1DF.to_numpy().flatten(order='F')
    QArray = np.quantile( npFlat, OurQs )
    H1StatsList.extend( QArray.tolist() )
    H1StatsList.append( npFlat.mean() )
    H1StatsList.append( npFlat.var() )
    H1StatsList.append( sstats.skew( npFlat ) )
    H1StatsList.append( sstats.kurtosis( npFlat ) )
    # do probabilistic time history
    DataDict = dict()
    for iI in range(NumQs):
        cQ = OurQs[iI]
        Hdr = "%d_ptile" % round(cQ * 100.0)
        DataDict[Hdr] = np.array( H1DF.quantile(q=cQ, axis=1), dtype=np.float32 )
    # end of for
    QuantsDF = pd.DataFrame( index=H1DF.index, data=DataDict )
    QuantsDF["Average"] = H1DF.mean(axis=1)
    QuantsDF["Variance"] = H1DF.var(axis=1)
    QuantsDF["Skew"] = H1DF.skew(axis=1)
    QuantsDF["Kurt"] = H1DF.kurtosis(axis=1)
    # now do day of the year distributions
    H1DF["IDayYear"] = H1DF.index.dayofyear
    DYList = list()
    for dD in DayYrList:
        LocalList = list()
        cDyOYrDF = H1DF[H1DF["IDayYear"] == dD].copy()
        cDyOYrDF.drop(columns="IDayYear", inplace=True )
        npFlat = cDyOYrDF.to_numpy().flatten(order='F')
        QArray = np.quantile( npFlat, OurQs )
        LocalList.extend( QArray.tolist() )
        LocalList.append( npFlat.mean() )
        LocalList.append( npFlat.var() )
        LocalList.append( sstats.skew( npFlat ) )
        LocalList.append( sstats.kurtosis( npFlat ) )
        DYList.append( LocalList )
    # end of for
    DataDict = dict()
    iCnt = 0
    for tHdr in StatsHdsList:
        DataDict[tHdr] = np.array( [ x[iCnt] for x in DYList ], dtype=np.float32 )
        iCnt += 1
    # end of for
    DailyQStatsDF = pd.DataFrame( index=DayYrIndex, data=DataDict )
    H1DF.drop( columns="IDayYear", inplace=True )
    # make a copy of H1DF for further manipulation
    H1DF["Month"] = H1DF.index.month
    PTH1DF = H1DF.copy()
    PTH1DF["Year"] = PTH1DF.index.year
    PTH1DF["Average"] = PTH1DF[RealColumns].mean(axis=1)
    PTH1DF["Max"] = PTH1DF[RealColumns].max(axis=1)
    PTH1DF["Min"] = PTH1DF[RealColumns].min(axis=1)
    PTH1DF.drop(columns=RealColumns, inplace=True)
    H1grouped = PTH1DF.groupby( ['Year', 'Month'], as_index=True )
    H1TSOutMeanDF = H1grouped[["Average"]].aggregate( [np.mean] )
    H1TSOutMaxDF = H1grouped[["Max"]].aggregate( [np.max] )
    H1TSOutMinDF = H1grouped[["Min"]].aggregate( [np.min] )
    PTMonthly = H1TSOutMaxDF.merge( H1TSOutMeanDF, how='inner', left_index=True, right_index=True )
    PTMonthly = PTMonthly.merge( H1TSOutMinDF, how='inner', left_index=True, right_index=True )
    # split into months
    H1Jan = H1DF[H1DF["Month"] == 1].copy()
    H1Jan.drop(columns="Month", inplace=True)
    H1Feb = H1DF[H1DF["Month"] == 2].copy()
    H1Feb.drop(columns="Month", inplace=True)
    H1Mar = H1DF[H1DF["Month"] == 3].copy()
    H1Mar.drop(columns="Month", inplace=True)
    H1Apr = H1DF[H1DF["Month"] == 4].copy()
    H1Apr.drop(columns="Month", inplace=True)
    H1May = H1DF[H1DF["Month"] == 5].copy()
    H1May.drop(columns="Month", inplace=True)
    H1Jun = H1DF[H1DF["Month"] == 6].copy()
    H1Jun.drop(columns="Month", inplace=True)
    H1Jul = H1DF[H1DF["Month"] == 7].copy()
    H1Jul.drop(columns="Month", inplace=True)
    H1Aug = H1DF[H1DF["Month"] == 8].copy()
    H1Aug.drop(columns="Month", inplace=True)
    H1Sep = H1DF[H1DF["Month"] == 9].copy()
    H1Sep.drop(columns="Month", inplace=True)
    H1Oct = H1DF[H1DF["Month"] == 10].copy()
    H1Oct.drop(columns="Month", inplace=True)
    H1Nov = H1DF[H1DF["Month"] == 11].copy()
    H1Nov.drop(columns="Month", inplace=True)
    H1Dec = H1DF[H1DF["Month"] == 12].copy()
    H1Dec.drop(columns="Month", inplace=True)
    MonAggList = list()
    # Jan
    LocalList = list()
    npFlat = H1Jan.to_numpy().flatten(order='F')
    QArray = np.quantile( npFlat, OurQs )
    LocalList.extend( QArray.tolist() )
    LocalList.append( npFlat.mean() )
    LocalList.append( npFlat.var() )
    LocalList.append( sstats.skew( npFlat ) )
    LocalList.append( sstats.kurtosis( npFlat ) )
    MonAggList.append( LocalList )
    # Feb
    LocalList = list()
    npFlat = H1Feb.to_numpy().flatten(order='F')
    QArray = np.quantile( npFlat, OurQs )
    LocalList.extend( QArray.tolist() )
    LocalList.append( npFlat.mean() )
    LocalList.append( npFlat.var() )
    LocalList.append( sstats.skew( npFlat ) )
    LocalList.append( sstats.kurtosis( npFlat ) )
    MonAggList.append( LocalList )
    # Mar
    LocalList = list()
    npFlat = H1Mar.to_numpy().flatten(order='F')
    QArray = np.quantile( npFlat, OurQs )
    LocalList.extend( QArray.tolist() )
    LocalList.append( npFlat.mean() )
    LocalList.append( npFlat.var() )
    LocalList.append( sstats.skew( npFlat ) )
    LocalList.append( sstats.kurtosis( npFlat ) )
    MonAggList.append( LocalList )
    # Apr
    LocalList = list()
    npFlat = H1Apr.to_numpy().flatten(order='F')
    QArray = np.quantile( npFlat, OurQs )
    LocalList.extend( QArray.tolist() )
    LocalList.append( npFlat.mean() )
    LocalList.append( npFlat.var() )
    LocalList.append( sstats.skew( npFlat ) )
    LocalList.append( sstats.kurtosis( npFlat ) )
    MonAggList.append( LocalList )
    # May
    LocalList = list()
    npFlat = H1May.to_numpy().flatten(order='F')
    QArray = np.quantile( npFlat, OurQs )
    LocalList.extend( QArray.tolist() )
    LocalList.append( npFlat.mean() )
    LocalList.append( npFlat.var() )
    LocalList.append( sstats.skew( npFlat ) )
    LocalList.append( sstats.kurtosis( npFlat ) )
    MonAggList.append( LocalList )
    # June
    LocalList = list()
    npFlat = H1Jun.to_numpy().flatten(order='F')
    QArray = np.quantile( npFlat, OurQs )
    LocalList.extend( QArray.tolist() )
    LocalList.append( npFlat.mean() )
    LocalList.append( npFlat.var() )
    LocalList.append( sstats.skew( npFlat ) )
    LocalList.append( sstats.kurtosis( npFlat ) )
    MonAggList.append( LocalList )
    # Jul
    LocalList = list()
    npFlat = H1Jul.to_numpy().flatten(order='F')
    QArray = np.quantile( npFlat, OurQs )
    LocalList.extend( QArray.tolist() )
    LocalList.append( npFlat.mean() )
    LocalList.append( npFlat.var() )
    LocalList.append( sstats.skew( npFlat ) )
    LocalList.append( sstats.kurtosis( npFlat ) )
    MonAggList.append( LocalList )
    # Aug
    LocalList = list()
    npFlat = H1Aug.to_numpy().flatten(order='F')
    QArray = np.quantile( npFlat, OurQs )
    LocalList.extend( QArray.tolist() )
    LocalList.append( npFlat.mean() )
    LocalList.append( npFlat.var() )
    LocalList.append( sstats.skew( npFlat ) )
    LocalList.append( sstats.kurtosis( npFlat ) )
    MonAggList.append( LocalList )
    # Sep
    LocalList = list()
    npFlat = H1Sep.to_numpy().flatten(order='F')
    QArray = np.quantile( npFlat, OurQs )
    LocalList.extend( QArray.tolist() )
    LocalList.append( npFlat.mean() )
    LocalList.append( npFlat.var() )
    LocalList.append( sstats.skew( npFlat ) )
    LocalList.append( sstats.kurtosis( npFlat ) )
    MonAggList.append( LocalList )
    # Oct
    LocalList = list()
    npFlat = H1Oct.to_numpy().flatten(order='F')
    QArray = np.quantile( npFlat, OurQs )
    LocalList.extend( QArray.tolist() )
    LocalList.append( npFlat.mean() )
    LocalList.append( npFlat.var() )
    LocalList.append( sstats.skew( npFlat ) )
    LocalList.append( sstats.kurtosis( npFlat ) )
    MonAggList.append( LocalList )
    # Nov
    LocalList = list()
    npFlat = H1Nov.to_numpy().flatten(order='F')
    QArray = np.quantile( npFlat, OurQs )
    LocalList.extend( QArray.tolist() )
    LocalList.append( npFlat.mean() )
    LocalList.append( npFlat.var() )
    LocalList.append( sstats.skew( npFlat ) )
    LocalList.append( sstats.kurtosis( npFlat ) )
    MonAggList.append( LocalList )
    # Dec
    LocalList = list()
    npFlat = H1Dec.to_numpy().flatten(order='F')
    QArray = np.quantile( npFlat, OurQs )
    LocalList.extend( QArray.tolist() )
    LocalList.append( npFlat.mean() )
    LocalList.append( npFlat.var() )
    LocalList.append( sstats.skew( npFlat ) )
    LocalList.append( sstats.kurtosis( npFlat ) )
    MonAggList.append( LocalList )
    MonAggList.append( H1StatsList )
    DataDict = dict()
    iCnt = 0
    for tHdr in StatsHdsList:
        DataDict[tHdr] = np.array( [ x[iCnt] for x in MonAggList ], dtype=np.float32 )
        iCnt += 1
    # end of for
    SummQStatsDF = pd.DataFrame( index=MonthLister, data=DataDict )
    # now write our worksheets
    OutFilePath = os.path.normpath( os.path.join( IN_DIR, "%s_H1CProj1_Precip.xlsx" % OUT_ROOT ) )
    if gCnt == 0:
        with pd.ExcelWriter( OutFilePath, engine="openpyxl" ) as writer:
            SummQStatsDF.to_excel( writer, sheet_name="Summary_%d" % gG, na_rep=str(np.nan), 
                                   columns=StatsHdsList )
            QuantsDF.to_excel( writer, sheet_name="Daily_Stats_%d" % gG, na_rep=str(np.nan), 
                               columns=StatsHdsList )
            DailyQStatsDF.to_excel( writer, sheet_name="DayYear_Stats_%d" % gG, na_rep=str(np.nan), 
                                    columns=StatsHdsList )
            PTMonthly.to_excel( writer, sheet_name="Monthly_Stats_%d" % gG, na_rep=str(np.nan) )
    else:
        with pd.ExcelWriter( OutFilePath, engine="openpyxl", mode='a' ) as writer:
            SummQStatsDF.to_excel( writer, sheet_name="Summary_%d" % gG, na_rep=str(np.nan), 
                                   columns=StatsHdsList )
            QuantsDF.to_excel( writer, sheet_name="Daily_Stats_%d" % gG, na_rep=str(np.nan), 
                               columns=StatsHdsList )
            DailyQStatsDF.to_excel( writer, sheet_name="DayYear_Stats_%d" % gG, na_rep=str(np.nan), 
                                    columns=StatsHdsList )
            PTMonthly.to_excel( writer, sheet_name="Monthly_Stats_%d" % gG, na_rep=str(np.nan) )
    # increment our counter
    gCnt += 1

### H1 CProj Period 2

Expect all of the H1 projection cases to be different.

In [13]:
gCnt = 0
for gG in WGI.LOCA_KEYS:
    InFileName = "%s%d.%s" % ( H1P2_ROOT, gG, PICKLE_EXT )
    InFile = os.path.normpath( os.path.join( IN_DIR, InFileName ) )
    H1DF = pd.read_pickle( InFile, compression='zip' )
    # collect the overall statistics
    H1StatsList = list()
    npFlat = H1DF.to_numpy().flatten(order='F')
    QArray = np.quantile( npFlat, OurQs )
    H1StatsList.extend( QArray.tolist() )
    H1StatsList.append( npFlat.mean() )
    H1StatsList.append( npFlat.var() )
    H1StatsList.append( sstats.skew( npFlat ) )
    H1StatsList.append( sstats.kurtosis( npFlat ) )
    # do probabilistic time history
    DataDict = dict()
    for iI in range(NumQs):
        cQ = OurQs[iI]
        Hdr = "%d_ptile" % round(cQ * 100.0)
        DataDict[Hdr] = np.array( H1DF.quantile(q=cQ, axis=1), dtype=np.float32 )
    # end of for
    QuantsDF = pd.DataFrame( index=H1DF.index, data=DataDict )
    QuantsDF["Average"] = H1DF.mean(axis=1)
    QuantsDF["Variance"] = H1DF.var(axis=1)
    QuantsDF["Skew"] = H1DF.skew(axis=1)
    QuantsDF["Kurt"] = H1DF.kurtosis(axis=1)
    # now do day of the year distributions
    H1DF["IDayYear"] = H1DF.index.dayofyear
    DYList = list()
    for dD in DayYrList:
        LocalList = list()
        cDyOYrDF = H1DF[H1DF["IDayYear"] == dD].copy()
        cDyOYrDF.drop(columns="IDayYear", inplace=True )
        npFlat = cDyOYrDF.to_numpy().flatten(order='F')
        QArray = np.quantile( npFlat, OurQs )
        LocalList.extend( QArray.tolist() )
        LocalList.append( npFlat.mean() )
        LocalList.append( npFlat.var() )
        LocalList.append( sstats.skew( npFlat ) )
        LocalList.append( sstats.kurtosis( npFlat ) )
        DYList.append( LocalList )
    # end of for
    DataDict = dict()
    iCnt = 0
    for tHdr in StatsHdsList:
        DataDict[tHdr] = np.array( [ x[iCnt] for x in DYList ], dtype=np.float32 )
        iCnt += 1
    # end of for
    DailyQStatsDF = pd.DataFrame( index=DayYrIndex, data=DataDict )
    H1DF.drop( columns="IDayYear", inplace=True )
    # make a copy of H1DF for further manipulation
    H1DF["Month"] = H1DF.index.month
    PTH1DF = H1DF.copy()
    PTH1DF["Year"] = PTH1DF.index.year
    PTH1DF["Average"] = PTH1DF[RealColumns].mean(axis=1)
    PTH1DF["Max"] = PTH1DF[RealColumns].max(axis=1)
    PTH1DF["Min"] = PTH1DF[RealColumns].min(axis=1)
    PTH1DF.drop(columns=RealColumns, inplace=True)
    H1grouped = PTH1DF.groupby( ['Year', 'Month'], as_index=True )
    H1TSOutMeanDF = H1grouped[["Average"]].aggregate( [np.mean] )
    H1TSOutMaxDF = H1grouped[["Max"]].aggregate( [np.max] )
    H1TSOutMinDF = H1grouped[["Min"]].aggregate( [np.min] )
    PTMonthly = H1TSOutMaxDF.merge( H1TSOutMeanDF, how='inner', left_index=True, right_index=True )
    PTMonthly = PTMonthly.merge( H1TSOutMinDF, how='inner', left_index=True, right_index=True )
    # split into months
    H1Jan = H1DF[H1DF["Month"] == 1].copy()
    H1Jan.drop(columns="Month", inplace=True)
    H1Feb = H1DF[H1DF["Month"] == 2].copy()
    H1Feb.drop(columns="Month", inplace=True)
    H1Mar = H1DF[H1DF["Month"] == 3].copy()
    H1Mar.drop(columns="Month", inplace=True)
    H1Apr = H1DF[H1DF["Month"] == 4].copy()
    H1Apr.drop(columns="Month", inplace=True)
    H1May = H1DF[H1DF["Month"] == 5].copy()
    H1May.drop(columns="Month", inplace=True)
    H1Jun = H1DF[H1DF["Month"] == 6].copy()
    H1Jun.drop(columns="Month", inplace=True)
    H1Jul = H1DF[H1DF["Month"] == 7].copy()
    H1Jul.drop(columns="Month", inplace=True)
    H1Aug = H1DF[H1DF["Month"] == 8].copy()
    H1Aug.drop(columns="Month", inplace=True)
    H1Sep = H1DF[H1DF["Month"] == 9].copy()
    H1Sep.drop(columns="Month", inplace=True)
    H1Oct = H1DF[H1DF["Month"] == 10].copy()
    H1Oct.drop(columns="Month", inplace=True)
    H1Nov = H1DF[H1DF["Month"] == 11].copy()
    H1Nov.drop(columns="Month", inplace=True)
    H1Dec = H1DF[H1DF["Month"] == 12].copy()
    H1Dec.drop(columns="Month", inplace=True)
    MonAggList = list()
    # Jan
    LocalList = list()
    npFlat = H1Jan.to_numpy().flatten(order='F')
    QArray = np.quantile( npFlat, OurQs )
    LocalList.extend( QArray.tolist() )
    LocalList.append( npFlat.mean() )
    LocalList.append( npFlat.var() )
    LocalList.append( sstats.skew( npFlat ) )
    LocalList.append( sstats.kurtosis( npFlat ) )
    MonAggList.append( LocalList )
    # Feb
    LocalList = list()
    npFlat = H1Feb.to_numpy().flatten(order='F')
    QArray = np.quantile( npFlat, OurQs )
    LocalList.extend( QArray.tolist() )
    LocalList.append( npFlat.mean() )
    LocalList.append( npFlat.var() )
    LocalList.append( sstats.skew( npFlat ) )
    LocalList.append( sstats.kurtosis( npFlat ) )
    MonAggList.append( LocalList )
    # Mar
    LocalList = list()
    npFlat = H1Mar.to_numpy().flatten(order='F')
    QArray = np.quantile( npFlat, OurQs )
    LocalList.extend( QArray.tolist() )
    LocalList.append( npFlat.mean() )
    LocalList.append( npFlat.var() )
    LocalList.append( sstats.skew( npFlat ) )
    LocalList.append( sstats.kurtosis( npFlat ) )
    MonAggList.append( LocalList )
    # Apr
    LocalList = list()
    npFlat = H1Apr.to_numpy().flatten(order='F')
    QArray = np.quantile( npFlat, OurQs )
    LocalList.extend( QArray.tolist() )
    LocalList.append( npFlat.mean() )
    LocalList.append( npFlat.var() )
    LocalList.append( sstats.skew( npFlat ) )
    LocalList.append( sstats.kurtosis( npFlat ) )
    MonAggList.append( LocalList )
    # May
    LocalList = list()
    npFlat = H1May.to_numpy().flatten(order='F')
    QArray = np.quantile( npFlat, OurQs )
    LocalList.extend( QArray.tolist() )
    LocalList.append( npFlat.mean() )
    LocalList.append( npFlat.var() )
    LocalList.append( sstats.skew( npFlat ) )
    LocalList.append( sstats.kurtosis( npFlat ) )
    MonAggList.append( LocalList )
    # June
    LocalList = list()
    npFlat = H1Jun.to_numpy().flatten(order='F')
    QArray = np.quantile( npFlat, OurQs )
    LocalList.extend( QArray.tolist() )
    LocalList.append( npFlat.mean() )
    LocalList.append( npFlat.var() )
    LocalList.append( sstats.skew( npFlat ) )
    LocalList.append( sstats.kurtosis( npFlat ) )
    MonAggList.append( LocalList )
    # Jul
    LocalList = list()
    npFlat = H1Jul.to_numpy().flatten(order='F')
    QArray = np.quantile( npFlat, OurQs )
    LocalList.extend( QArray.tolist() )
    LocalList.append( npFlat.mean() )
    LocalList.append( npFlat.var() )
    LocalList.append( sstats.skew( npFlat ) )
    LocalList.append( sstats.kurtosis( npFlat ) )
    MonAggList.append( LocalList )
    # Aug
    LocalList = list()
    npFlat = H1Aug.to_numpy().flatten(order='F')
    QArray = np.quantile( npFlat, OurQs )
    LocalList.extend( QArray.tolist() )
    LocalList.append( npFlat.mean() )
    LocalList.append( npFlat.var() )
    LocalList.append( sstats.skew( npFlat ) )
    LocalList.append( sstats.kurtosis( npFlat ) )
    MonAggList.append( LocalList )
    # Sep
    LocalList = list()
    npFlat = H1Sep.to_numpy().flatten(order='F')
    QArray = np.quantile( npFlat, OurQs )
    LocalList.extend( QArray.tolist() )
    LocalList.append( npFlat.mean() )
    LocalList.append( npFlat.var() )
    LocalList.append( sstats.skew( npFlat ) )
    LocalList.append( sstats.kurtosis( npFlat ) )
    MonAggList.append( LocalList )
    # Oct
    LocalList = list()
    npFlat = H1Oct.to_numpy().flatten(order='F')
    QArray = np.quantile( npFlat, OurQs )
    LocalList.extend( QArray.tolist() )
    LocalList.append( npFlat.mean() )
    LocalList.append( npFlat.var() )
    LocalList.append( sstats.skew( npFlat ) )
    LocalList.append( sstats.kurtosis( npFlat ) )
    MonAggList.append( LocalList )
    # Nov
    LocalList = list()
    npFlat = H1Nov.to_numpy().flatten(order='F')
    QArray = np.quantile( npFlat, OurQs )
    LocalList.extend( QArray.tolist() )
    LocalList.append( npFlat.mean() )
    LocalList.append( npFlat.var() )
    LocalList.append( sstats.skew( npFlat ) )
    LocalList.append( sstats.kurtosis( npFlat ) )
    MonAggList.append( LocalList )
    # Dec
    LocalList = list()
    npFlat = H1Dec.to_numpy().flatten(order='F')
    QArray = np.quantile( npFlat, OurQs )
    LocalList.extend( QArray.tolist() )
    LocalList.append( npFlat.mean() )
    LocalList.append( npFlat.var() )
    LocalList.append( sstats.skew( npFlat ) )
    LocalList.append( sstats.kurtosis( npFlat ) )
    MonAggList.append( LocalList )
    MonAggList.append( H1StatsList )
    DataDict = dict()
    iCnt = 0
    for tHdr in StatsHdsList:
        DataDict[tHdr] = np.array( [ x[iCnt] for x in MonAggList ], dtype=np.float32 )
        iCnt += 1
    # end of for
    SummQStatsDF = pd.DataFrame( index=MonthLister, data=DataDict )
    # now write our worksheets
    OutFilePath = os.path.normpath( os.path.join( IN_DIR, "%s_H1CProj2_Precip.xlsx" % OUT_ROOT ) )
    if gCnt == 0:
        with pd.ExcelWriter( OutFilePath, engine="openpyxl" ) as writer:
            SummQStatsDF.to_excel( writer, sheet_name="Summary_%d" % gG, na_rep=str(np.nan), 
                                   columns=StatsHdsList )
            QuantsDF.to_excel( writer, sheet_name="Daily_Stats_%d" % gG, na_rep=str(np.nan), 
                               columns=StatsHdsList )
            DailyQStatsDF.to_excel( writer, sheet_name="DayYear_Stats_%d" % gG, na_rep=str(np.nan), 
                                    columns=StatsHdsList )
            PTMonthly.to_excel( writer, sheet_name="Monthly_Stats_%d" % gG, na_rep=str(np.nan) )
    else:
        with pd.ExcelWriter( OutFilePath, engine="openpyxl", mode='a' ) as writer:
            SummQStatsDF.to_excel( writer, sheet_name="Summary_%d" % gG, na_rep=str(np.nan), 
                                   columns=StatsHdsList )
            QuantsDF.to_excel( writer, sheet_name="Daily_Stats_%d" % gG, na_rep=str(np.nan), 
                               columns=StatsHdsList )
            DailyQStatsDF.to_excel( writer, sheet_name="DayYear_Stats_%d" % gG, na_rep=str(np.nan), 
                                    columns=StatsHdsList )
            PTMonthly.to_excel( writer, sheet_name="Monthly_Stats_%d" % gG, na_rep=str(np.nan) )
    # increment our counter
    gCnt += 1

### H1 CProj Period 3

Expect all of the H1 projection cases to be different.

In [14]:
gCnt = 0
for gG in WGI.LOCA_KEYS:
    InFileName = "%s%d.%s" % ( H1P3_ROOT, gG, PICKLE_EXT )
    InFile = os.path.normpath( os.path.join( IN_DIR, InFileName ) )
    H1DF = pd.read_pickle( InFile, compression='zip' )
    # collect the overall statistics
    H1StatsList = list()
    npFlat = H1DF.to_numpy().flatten(order='F')
    QArray = np.quantile( npFlat, OurQs )
    H1StatsList.extend( QArray.tolist() )
    H1StatsList.append( npFlat.mean() )
    H1StatsList.append( npFlat.var() )
    H1StatsList.append( sstats.skew( npFlat ) )
    H1StatsList.append( sstats.kurtosis( npFlat ) )
    # do probabilistic time history
    DataDict = dict()
    for iI in range(NumQs):
        cQ = OurQs[iI]
        Hdr = "%d_ptile" % round(cQ * 100.0)
        DataDict[Hdr] = np.array( H1DF.quantile(q=cQ, axis=1), dtype=np.float32 )
    # end of for
    QuantsDF = pd.DataFrame( index=H1DF.index, data=DataDict )
    QuantsDF["Average"] = H1DF.mean(axis=1)
    QuantsDF["Variance"] = H1DF.var(axis=1)
    QuantsDF["Skew"] = H1DF.skew(axis=1)
    QuantsDF["Kurt"] = H1DF.kurtosis(axis=1)
    # now do day of the year distributions
    H1DF["IDayYear"] = H1DF.index.dayofyear
    DYList = list()
    for dD in DayYrList:
        LocalList = list()
        cDyOYrDF = H1DF[H1DF["IDayYear"] == dD].copy()
        cDyOYrDF.drop(columns="IDayYear", inplace=True )
        npFlat = cDyOYrDF.to_numpy().flatten(order='F')
        QArray = np.quantile( npFlat, OurQs )
        LocalList.extend( QArray.tolist() )
        LocalList.append( npFlat.mean() )
        LocalList.append( npFlat.var() )
        LocalList.append( sstats.skew( npFlat ) )
        LocalList.append( sstats.kurtosis( npFlat ) )
        DYList.append( LocalList )
    # end of for
    DataDict = dict()
    iCnt = 0
    for tHdr in StatsHdsList:
        DataDict[tHdr] = np.array( [ x[iCnt] for x in DYList ], dtype=np.float32 )
        iCnt += 1
    # end of for
    DailyQStatsDF = pd.DataFrame( index=DayYrIndex, data=DataDict )
    H1DF.drop( columns="IDayYear", inplace=True )
    # make a copy of H1DF for further manipulation
    H1DF["Month"] = H1DF.index.month
    PTH1DF = H1DF.copy()
    PTH1DF["Year"] = PTH1DF.index.year
    PTH1DF["Average"] = PTH1DF[RealColumns].mean(axis=1)
    PTH1DF["Max"] = PTH1DF[RealColumns].max(axis=1)
    PTH1DF["Min"] = PTH1DF[RealColumns].min(axis=1)
    PTH1DF.drop(columns=RealColumns, inplace=True)
    H1grouped = PTH1DF.groupby( ['Year', 'Month'], as_index=True )
    H1TSOutMeanDF = H1grouped[["Average"]].aggregate( [np.mean] )
    H1TSOutMaxDF = H1grouped[["Max"]].aggregate( [np.max] )
    H1TSOutMinDF = H1grouped[["Min"]].aggregate( [np.min] )
    PTMonthly = H1TSOutMaxDF.merge( H1TSOutMeanDF, how='inner', left_index=True, right_index=True )
    PTMonthly = PTMonthly.merge( H1TSOutMinDF, how='inner', left_index=True, right_index=True )
    # split into months
    H1Jan = H1DF[H1DF["Month"] == 1].copy()
    H1Jan.drop(columns="Month", inplace=True)
    H1Feb = H1DF[H1DF["Month"] == 2].copy()
    H1Feb.drop(columns="Month", inplace=True)
    H1Mar = H1DF[H1DF["Month"] == 3].copy()
    H1Mar.drop(columns="Month", inplace=True)
    H1Apr = H1DF[H1DF["Month"] == 4].copy()
    H1Apr.drop(columns="Month", inplace=True)
    H1May = H1DF[H1DF["Month"] == 5].copy()
    H1May.drop(columns="Month", inplace=True)
    H1Jun = H1DF[H1DF["Month"] == 6].copy()
    H1Jun.drop(columns="Month", inplace=True)
    H1Jul = H1DF[H1DF["Month"] == 7].copy()
    H1Jul.drop(columns="Month", inplace=True)
    H1Aug = H1DF[H1DF["Month"] == 8].copy()
    H1Aug.drop(columns="Month", inplace=True)
    H1Sep = H1DF[H1DF["Month"] == 9].copy()
    H1Sep.drop(columns="Month", inplace=True)
    H1Oct = H1DF[H1DF["Month"] == 10].copy()
    H1Oct.drop(columns="Month", inplace=True)
    H1Nov = H1DF[H1DF["Month"] == 11].copy()
    H1Nov.drop(columns="Month", inplace=True)
    H1Dec = H1DF[H1DF["Month"] == 12].copy()
    H1Dec.drop(columns="Month", inplace=True)
    MonAggList = list()
    # Jan
    LocalList = list()
    npFlat = H1Jan.to_numpy().flatten(order='F')
    QArray = np.quantile( npFlat, OurQs )
    LocalList.extend( QArray.tolist() )
    LocalList.append( npFlat.mean() )
    LocalList.append( npFlat.var() )
    LocalList.append( sstats.skew( npFlat ) )
    LocalList.append( sstats.kurtosis( npFlat ) )
    MonAggList.append( LocalList )
    # Feb
    LocalList = list()
    npFlat = H1Feb.to_numpy().flatten(order='F')
    QArray = np.quantile( npFlat, OurQs )
    LocalList.extend( QArray.tolist() )
    LocalList.append( npFlat.mean() )
    LocalList.append( npFlat.var() )
    LocalList.append( sstats.skew( npFlat ) )
    LocalList.append( sstats.kurtosis( npFlat ) )
    MonAggList.append( LocalList )
    # Mar
    LocalList = list()
    npFlat = H1Mar.to_numpy().flatten(order='F')
    QArray = np.quantile( npFlat, OurQs )
    LocalList.extend( QArray.tolist() )
    LocalList.append( npFlat.mean() )
    LocalList.append( npFlat.var() )
    LocalList.append( sstats.skew( npFlat ) )
    LocalList.append( sstats.kurtosis( npFlat ) )
    MonAggList.append( LocalList )
    # Apr
    LocalList = list()
    npFlat = H1Apr.to_numpy().flatten(order='F')
    QArray = np.quantile( npFlat, OurQs )
    LocalList.extend( QArray.tolist() )
    LocalList.append( npFlat.mean() )
    LocalList.append( npFlat.var() )
    LocalList.append( sstats.skew( npFlat ) )
    LocalList.append( sstats.kurtosis( npFlat ) )
    MonAggList.append( LocalList )
    # May
    LocalList = list()
    npFlat = H1May.to_numpy().flatten(order='F')
    QArray = np.quantile( npFlat, OurQs )
    LocalList.extend( QArray.tolist() )
    LocalList.append( npFlat.mean() )
    LocalList.append( npFlat.var() )
    LocalList.append( sstats.skew( npFlat ) )
    LocalList.append( sstats.kurtosis( npFlat ) )
    MonAggList.append( LocalList )
    # June
    LocalList = list()
    npFlat = H1Jun.to_numpy().flatten(order='F')
    QArray = np.quantile( npFlat, OurQs )
    LocalList.extend( QArray.tolist() )
    LocalList.append( npFlat.mean() )
    LocalList.append( npFlat.var() )
    LocalList.append( sstats.skew( npFlat ) )
    LocalList.append( sstats.kurtosis( npFlat ) )
    MonAggList.append( LocalList )
    # Jul
    LocalList = list()
    npFlat = H1Jul.to_numpy().flatten(order='F')
    QArray = np.quantile( npFlat, OurQs )
    LocalList.extend( QArray.tolist() )
    LocalList.append( npFlat.mean() )
    LocalList.append( npFlat.var() )
    LocalList.append( sstats.skew( npFlat ) )
    LocalList.append( sstats.kurtosis( npFlat ) )
    MonAggList.append( LocalList )
    # Aug
    LocalList = list()
    npFlat = H1Aug.to_numpy().flatten(order='F')
    QArray = np.quantile( npFlat, OurQs )
    LocalList.extend( QArray.tolist() )
    LocalList.append( npFlat.mean() )
    LocalList.append( npFlat.var() )
    LocalList.append( sstats.skew( npFlat ) )
    LocalList.append( sstats.kurtosis( npFlat ) )
    MonAggList.append( LocalList )
    # Sep
    LocalList = list()
    npFlat = H1Sep.to_numpy().flatten(order='F')
    QArray = np.quantile( npFlat, OurQs )
    LocalList.extend( QArray.tolist() )
    LocalList.append( npFlat.mean() )
    LocalList.append( npFlat.var() )
    LocalList.append( sstats.skew( npFlat ) )
    LocalList.append( sstats.kurtosis( npFlat ) )
    MonAggList.append( LocalList )
    # Oct
    LocalList = list()
    npFlat = H1Oct.to_numpy().flatten(order='F')
    QArray = np.quantile( npFlat, OurQs )
    LocalList.extend( QArray.tolist() )
    LocalList.append( npFlat.mean() )
    LocalList.append( npFlat.var() )
    LocalList.append( sstats.skew( npFlat ) )
    LocalList.append( sstats.kurtosis( npFlat ) )
    MonAggList.append( LocalList )
    # Nov
    LocalList = list()
    npFlat = H1Nov.to_numpy().flatten(order='F')
    QArray = np.quantile( npFlat, OurQs )
    LocalList.extend( QArray.tolist() )
    LocalList.append( npFlat.mean() )
    LocalList.append( npFlat.var() )
    LocalList.append( sstats.skew( npFlat ) )
    LocalList.append( sstats.kurtosis( npFlat ) )
    MonAggList.append( LocalList )
    # Dec
    LocalList = list()
    npFlat = H1Dec.to_numpy().flatten(order='F')
    QArray = np.quantile( npFlat, OurQs )
    LocalList.extend( QArray.tolist() )
    LocalList.append( npFlat.mean() )
    LocalList.append( npFlat.var() )
    LocalList.append( sstats.skew( npFlat ) )
    LocalList.append( sstats.kurtosis( npFlat ) )
    MonAggList.append( LocalList )
    MonAggList.append( H1StatsList )
    DataDict = dict()
    iCnt = 0
    for tHdr in StatsHdsList:
        DataDict[tHdr] = np.array( [ x[iCnt] for x in MonAggList ], dtype=np.float32 )
        iCnt += 1
    # end of for
    SummQStatsDF = pd.DataFrame( index=MonthLister, data=DataDict )
    # now write our worksheets
    OutFilePath = os.path.normpath( os.path.join( IN_DIR, "%s_H1CProj3_Precip.xlsx" % OUT_ROOT ) )
    if gCnt == 0:
        with pd.ExcelWriter( OutFilePath, engine="openpyxl" ) as writer:
            SummQStatsDF.to_excel( writer, sheet_name="Summary_%d" % gG, na_rep=str(np.nan), 
                                   columns=StatsHdsList )
            QuantsDF.to_excel( writer, sheet_name="Daily_Stats_%d" % gG, na_rep=str(np.nan), 
                               columns=StatsHdsList )
            DailyQStatsDF.to_excel( writer, sheet_name="DayYear_Stats_%d" % gG, na_rep=str(np.nan), 
                                    columns=StatsHdsList )
            PTMonthly.to_excel( writer, sheet_name="Monthly_Stats_%d" % gG, na_rep=str(np.nan) )
    else:
        with pd.ExcelWriter( OutFilePath, engine="openpyxl", mode='a' ) as writer:
            SummQStatsDF.to_excel( writer, sheet_name="Summary_%d" % gG, na_rep=str(np.nan), 
                                   columns=StatsHdsList )
            QuantsDF.to_excel( writer, sheet_name="Daily_Stats_%d" % gG, na_rep=str(np.nan), 
                               columns=StatsHdsList )
            DailyQStatsDF.to_excel( writer, sheet_name="DayYear_Stats_%d" % gG, na_rep=str(np.nan), 
                                    columns=StatsHdsList )
            PTMonthly.to_excel( writer, sheet_name="Monthly_Stats_%d" % gG, na_rep=str(np.nan) )
    # increment our counter
    gCnt += 1